<a href="https://colab.research.google.com/github/mgaac/paper_imlementations/blob/main/MultiHeadAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as f

import math

In [156]:
class MultiHeadAttention(nn.Module):
  def __init__(self, input_size, embed_size, n_heads):
    super(MultiHeadAttention, self).__init__()

    self.embed_size = embed_size
    self.input_size = input_size
    self.n_heads = n_heads

    self.embedding = nn.Linear(input_size, embed_size)

    self.query = nn.Linear(embed_size, embed_size * n_heads)
    self.key = nn.Linear(embed_size, embed_size * n_heads)
    self.value = nn.Linear(embed_size, embed_size * n_heads)

  def forward(self, x):
    embed_size = self.embed_size 
    input_size = self.input_size
    n_heads = self.n_heads

    x = self.embedding(x)

    query = self.query(x)
    key = self.key(x)
    value = self.value(x)

    query = query.reshape(x.size()[0], embed_size, n_heads)
    key = key.reshape(x.size()[0], embed_size, n_heads)
    value = value.reshape(x.size()[0], embed_size, n_heads)

    alingment = torch.matmul(query, torch.transpose(key, 1, 2))[:,0]
    weights = f.softmax(torch.div(alingment, math.sqrt(embed_size)), 1)
    weighted_values = torch.einsum('ikj, ik->ikj', [value,weights])

    return weighted_values

MultiHeadAttention = MultiHeadAttention(1, 10, 10)

In [159]:
input = torch.rand(200,1)
MultiHeadAttention(input).size()

torch.Size([200, 10, 10])